In [1]:
import pandas as pd
import re 
import string

import io 
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

import pdfplumber

import requests

/home/noahdarwich/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [92]:
import tempfile 
from oauth2client.service_account import ServiceAccountCredentials

from googleapiclient.http import MediaFileUpload

from googleapiclient.http import MediaIoBaseDownload

In [3]:
from arabic_reshaper import ArabicReshaper
from bidi.algorithm import get_display

In [4]:
from camel_tools.utils.dediac import dediac_ar
from ar_corrector.corrector import Corrector

In [25]:
import PyPDF2

from pdfquery import PDFQuery
from io import BytesIO, StringIO
from PIL import Image

import pytesseract

from pdfminer.high_level import extract_text_to_fp
from pdfminer.layout import LAParams
from PyPDF2 import PdfFileReader, PdfFileWriter
import pyarabic.araby as araby

import camelot

from unidecode import unidecode

**Extract text from pdf**

In [6]:
def extract_text_from_pdf(pdf_content):
  resource_manager = PDFResourceManager()
  fake_file_handle = io.StringIO()
  converter = TextConverter(resource_manager, fake_file_handle)
  page_interpreter = PDFPageInterpreter(resource_manager, converter)

  with io.BytesIO(pdf_content) as fh:
    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  check_extractable=True):
      page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

  # close open handles
  converter.close()
  fake_file_handle.close()

  return text

**Save pdf pages in Google Drive**

In [91]:
events_folder_id_dict = {'2010':'1CPmsZoNqsDslYUDuMMYAiTRdgcbjuIj6','2011':'1FQenIULvAsgQN4sLSpcLdjjfStfeIvga',
                       '2012':'1ABi7lAAmlARbrGlBJv8Tz6IFXVSYzLdR','2013':'1MMg8bUfZj3eZf_gAlr3k2N9_LWmO5DWr',
                       '2014':'135rCpr_1_N00nmsKOcyWjmQPmT_n8I96','2015':'1Gjm0qtTXL-21qNQJtloxNGtJq3Q1Xlyq',
                       '2016':'1qRUeOe4mFx5ofuYHxiqL4u1HvqxLCz96','2017':'15DY5sNvP8O2jgus7Qythy8Nzeeo7cZAI',
                       '2018':'1AKHSz-t3vkUsLE2IC4TdrMUznjNC3YpU','2019':'1qOX3DQFtU_j4czyUMndjpn9c0crZ8ue1'}
events_folder_id = '165-gELn98fyKGaWnmWSkJuUuz1aj8XMg'

In [ ]:
creds = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json',
                                                         scopes=['https://www.googleapis.com/auth/drive'])
service = build('drive', 'v3', credentials=creds)

def save_page(folder_id, folders_dic):
    pass

In [7]:
df_results = pd.read_csv('df_results.csv', index_col=0)
df_results

,issue,url,date,pdf_link,status
0,العدد 3922,https://www.al-akhbar.com/Editions/2019/12/2/PDF,2019-12-2,https://www.al-akhbar.com/PDF_Files/3922/alakh...,NaN
1,العدد 3923,https://www.al-akhbar.com/Editions/2019/12/3/PDF,2019-12-3,NaN,not_saved
2,العدد 3924,https://www.al-akhbar.com/Editions/2019/12/4/PDF,2019-12-4,https://www.al-akhbar.com/PDF_Files/3924/alakh...,NaN
3,العدد 3925,https://www.al-akhbar.com/Editions/2019/12/5/PDF,2019-12-5,https://www.al-akhbar.com/PDF_Files/3925/alakh...,NaN
4,العدد 3926,https://www.al-akhbar.com/Editions/2019/12/6/PDF,2019-12-6,https://www.al-akhbar.com/PDF_Files/3926/alakh...,NaN
...,...,...,...,...,...
2923,العدد 1028,https://www.al-akhbar.com/Editions/2010/1/26/PDF,2010-1-26,https://www.al-akhbar.com/PDF_Files/1028/alakh...,saved
2924,العدد 1029,https://www.al-akhbar.com/Editions/2010/1/27/PDF,2010-1-27,https://www.al-akhbar.com/PDF_Files/1029/alakh...,saved
2925,العدد 1030,https://www.al-akhbar.com/Editions/2010/1/28/PDF,2010-1-28,https://www.al-akhbar.com/PDF_Files/1030/alakh...,saved
2926,العدد 1031,https://www.al-akhbar.com/Editions/2010/1/29/PDF,2010-1-29,https://www.al-akhbar.com/PDF_Files/1031/alakh...,saved


**Create folder in Google Drive**

In [22]:
# creds = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json',
#                                                          scopes=['https://www.googleapis.com/auth/drive'])
# service = build('drive', 'v3', credentials=creds)
# folder_id = '1BVzI84DDDtbvU3pp1k0yERF5YKIs84R0'
# results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()

# files = results.get("files", [])
# next_page_token = results.get("nextPageToken")

# while next_page_token:
#     results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)", pageToken=next_page_token).execute()
#     files.extend(results.get("files", []))
#     next_page_token = results.get("nextPageToken")
    
# # Create new folder "keywords" inside "AAA"
# folder_metadata = {'name': 'events', 'parents': [folder_id], 'mimeType': 'application/vnd.google-apps.folder'}
# folder = service.files().create(body=folder_metadata).execute()
# events_folder_id = folder.get('id')

In [21]:
year_folder_id_dict = {'2010':'113DAB_g6P3b-gbrpFog682EfdILcBMt6','2011':'1Yh9610KUMxZLNDiyUOnng-7PbhlYbt3x',
                       '2012':'1TEWAyyxf6dJ3K6hMiliayJ4VV4ilvwIH','2013':'1KWNg6Me11spV-JRFOzwlvNpvoYMLI59C',
                       '2014':'1IgDuaPlc9b-MdP6VELFXcTffqSRHYPU5','2015':'1CrHueAEH76p95tIOryMrdu5nV-qW702M',
                       '2016':'1hpViY56I20A6k6EeQGrOn6QG11j7gb58','2017':'1H2HXwALMyzxBIMJyluYkTHJKplqGIpUF',
                       '2018':'1-jlTY-u_izedKKNenevC3kgM6yaKR7b7','2019':'1Qm-Sse4vbOx2IOAILUCTvXwqShWlynQn',}

**Read files in Google drive**

In [67]:
creds = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json',
                                                         scopes=['https://www.googleapis.com/auth/drive'])
service = build('drive', 'v3', credentials=creds)
folder_id = year_folder_id_dict['2016']

In [68]:
configuration = {
    'delete_harakat': True,
    'delete_tatweel ': True,
}

In [99]:
results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()
files = results.get('files', [])

# Iterate through the files
for file in files:
    file_id = file.get('id')
    file_name = file.get('name')
    
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        #print("Download %d%%." % int(status.progress() * 100))
    pdf_content = fh.getvalue()
    with open('new.pdf', 'wb') as f:
        f.write(pdf_content)
    
    pdf = PdfFileReader(open('new.pdf', 'rb'))
    num_pages = pdf.getNumPages()
    texts = {}
    for page in range(1,num_pages):
         with open('new.pdf', 'rb') as f:
            laparams = LAParams(char_margin=1.0, line_margin=0.5, word_margin=0.1)
            outfp = StringIO()
            outfp.seek(0)
            extract_text_to_fp(f, outfp, laparams=laparams, page_numbers={page})
            outfp.seek(0)
            text = outfp.read()
            reshaper = ArabicReshaper(configuration=configuration)
            corrected_text = get_display(text)
            reshaped_text = reshaper.reshape(corrected_text)
          
            section = ''.join(x for x in reshaped_text[:60] if x not in string.digits).strip().replace('\n',' ').replace('ـ', '')
            section = [x for x in section.split() if len(x) > 2]
           
            if 'ﻟﺒﻨﺎﻥ' in section or 'ﺳﻴﺎﺳﺔ' in section or 'ﻣﺠﺘﻤﻊ' in section or 'ﺍﻗﺘﺼﺎﺩ' in section or 'ﺇﻗﺘﺼﺎﺩ' in section:
                texts[page] = reshaped_text.strip().replace('\n',' ').replace('ـ', '').replace('�', '')
                output_pdf = PdfFileWriter()
                output_pdf.addPage(pdf.getPage(page))
                name = file_name.replace('.pdf', f'-page-{int(page)+1}.pdf')
                with open(name, 'wb') as f:
                    output_pdf.write(f)
                    file = tempfile.NamedTemporaryFile()
                    file.write(open(name, 'rb').read())
                    file.seek(0)
                    file_metadata = {'name': name,'parents':['1CPmsZoNqsDslYUDuMMYAiTRdgcbjuIj6']}
                    
                    media = MediaFileUpload(file.name,
                                    mimetype='application/pdf')
                    
                    service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    break

2016-1-30.pdf
Download 100%.


In [73]:
texts[1]

'ﺍﻟﺴﺒﺖ 30  ﻛﺎﻧﻮﻥ ﺍﻟﺜﺎﻧﻲ  2016  ﺍﻟﻌﺪﺩ  2801  ﺳﻴﺎﺳﺔ  2  ﻧﺼﺮﷲ »ﻳﻠﻢ « 8 ﺁﺫﺍﺭ: ﺍﻟﺮﺋﺎﺳﺔ ﻟﻨﺎ  ﻗﻀﻴﺔ ﺍﻟﻴﻮﻡ  ﺧﻄﺎﺏ ﺍﺍﻟﻤﻴﻦ ﺍﻟﻌﺎﻡ ﻟﺤﺰﺏ ﷲ ﺍﻟﺴﻴﺪ ﺣﺴﻦ ﻧﺼﺮﷲ ﺃﻣﺲ ﻫﺪﻑ ﺇﻟﻰ ﺗﺤﻘﻴﻖ ﺃﻣﺮﻳﻦ: ﺗﺄﻛﻴﺪ ﺍﻟﺘﻤﺴﻚ ﺑﺘﺮﺷﻴﺢ ﺍﻟﻌﻤﺎﺩ ﻣﻴﺸﺎﻝ  ﻋﻮﻥ، ﻭﺍﻟﺴﻌﻲ ﻣﻌﻪ ﻟﻠﻔﻮﺯ ﺑﺮﺋﺎﺳﺔ ﺍﻟﺠﻤﻬﻮﺭﻳﺔ؛ ﻭﺗﺨﻔﻴﻒ ﺍﻟﺘﺸﻨﺞ ﺑﻴﻦ ﻣﻜﻮ ﻧﺎﺕ ﺗﺤﺎﻟﻒ 8 ﺁﺫﺍﺭ  ﺍﻟﺘﻴﺎﺭ ﺍﻟﻮﻃﻨﻲ ﺍﻟﺤﺮ، ﻭﻓﺘﺢ ﺑﺎﺏ  ﺍﻟﺤﻮﺍﺭ ﺑﻴﻨﻬﺎ  ﻗﻄﻊ ﺍﺃﻟﻤﻦ ﺍﻟﻌﺎﻡ ﻟﺤﺰﺏ ﺍﻟﻠﻪ ﺍﻟﺴﻴﺪ  ﺣﺴﻦ ﻧﺼﺮﺍﻟﻠﻪ ﻛﻞ ﺍﻟﺸﻜﻮﻙ، ﻭﺃﻋﻠﻦ  ﻣﻮﻗﻒ ﺣﺰﺑﻪ ﻣﻦ ﺍﻣﻠﻠﻒ ﺍﻟﺮﺋﺎﺳﻲ:  ﻣﻠﺘﺰﻣﻮﻥ ﺗﺄﻳﻴﺪ ﺍﻟﻌﻤﺎﺩ ﻣﻴﺸﺎﻝ ﻋﻮﻥ،  ﻭﺍﻝ ﺗﺮﺍﺟﻊ ﻋﻦ ﻫﺬﺍ ﺍﺍﻟﻠﺘﺰﺍﻡ ﺇﺍﻝ ﻓﻲ  ﺣﺎﻟﺔ ﻭﺍﺣﺪﺓ، ﺍﻥ ﻱ ﻋﻠﻦ ﻋﻮﻥ ﺍﻧﺴﺤﺎﺑﻪ  ﻣﻦ ﺍﻟﺴﺒﺎﻕ ﺍﻟﺮﺋﺎﺳﻲ، ﻭﻓﻀﺎﻝ  ﻋﻦ ﺫﻟﻚ،  ﺟﺰﻡ ﻧﺼﺮ ﺍﻟﻠﻪ ﺑﺄﻥ ﺍﻟﺮﺋﻴﺲ ﺍﻣﻠﻘﺒﻞ  ﻟﻠﺠﻤﻬﻮﺭﻳﺔ ﻟﻦ ﻳﻜﻮﻥ ﺇﺍﻝ ﻣﻦ ﻓﺮﻳﻖ 8  ﺁﺫﺍﺭ. ﻭﺟﺰﻣﻪ ﻫﺬﺍ ﻣﺒﻨﻲ ﻋﻠﻰ ﻣﻮﻗﻒ ﻛﻞ  ﻣﻦ ﺗﻴﺎﺭ ﺍﻣﻠﺴﺘﻘﺒﻞ ﻭﺍﻟﻘﻮﺍﺕ ﺍﻟﻠﺒﻨﺎﻧﻴﺔ،   ﻣﻦ  ﺍﻟﻠﺬﻳﻦ ﺭﺵ ﺍﻟﻨﺎﺋﺐ ﺳﻠﻴﻤﺎﻥ ﻓﺮﻧﺠﻴﺔ ﻭﺍﻟﻌﻤﺎﺩ  ﻣﻴﺸﺎﻝ ﻋﻮﻥ، ﻭﺇﺿﺎﻓﺔ ﺇﻟﻰ ﺍﻣﻠﻮﻗﻒ  ﻭﺍﻟﻘﺮﺍﺀﺓ، ﺃﻃﻠﻖ ﻧﺼﺮﺍﻟﻠﻪ، ﺑﺨﻄﺎﺑﻪ  ﺍﻟﻬﺎﺩﺉ  ﻭﺍﺍﻟﺴﺘﻴﻌﺎﺑﻲ،  ﺣﻤﻠﺔ  »ﻣﻠﻠﻤﺔ«   ﺤﺎ ﺇﻟﻰ ﺍﻟﺮﺋﺎﺳﺔ ﻛﺎﻝ  ﺍﻝ ﻧﺮﻳﺪ ﺍﻟﺬﻫﺎﺏ ﺇﻟﻰ ﻣﺆﺗﻤﺮ ﺗﺄﺳﻴﺴﻲ ﻭﻣﺴﺘﻌﺪﻭﻥ ﻻﻟﻨﺘﺨﺎﺏ  ﻏﺪﺍ ﺗﺤﺖ ﺳﻘﻒ ﺍﻟﻄﺎﺋﻒ  ﺍﻟﻘﻮﻯ ﺍﻣﻠﻜﻮﻧﺔ ﻟﺘﺤﺎﻟﻒ 8 ﺁﺫﺍﺭ  ﺍﻟﺘﻴﺎﺭ  ﺍﻟﻮﻃﻨﻲ ﺍﻟﺤﺮ، ﻟﻴﻔﺘﺢ ﺍﻣﺎﻣﻬﺎ ﺃﺑﻮﺍﺏ  ﺍﻟﺤﻮﺍﺭ. ﻭ

**Move file in Google drive**

In [9]:
# # search for specific keywords in the content
# if 'keyword1' in content or 'keyword2' in content:
#     # move the file to the "keywords" folder
#     file_metadata = {'parents': [events_folder_id]}
#     service.files().update(fileId=file_id, body=file_metadata).execute()


**Correct Arabic Text**

In [10]:
# reshaper = ArabicReshaper(configuration=configuration)
# corrected_text = reshaper.reshape(text)
# corrected_text = get_display(text)
# corrected_text = corrected_text.replace('ـ', '')
# corrected_text

In [11]:
# from chardet.universaldetector import UniversalDetector
# detector = UniversalDetector()

In [8]:
test_sample = df_results[0:666]
test_folder_id = '19Nx9P1ekaDyoX06z2sj0FKokfcskouNc'

In [9]:
configuration = {
    'delete_harakat': True,
    'delete_tatweel ': True,
}
url = test_sample.iloc[222]['pdf_link']
url

'https://www.al-akhbar.com/PDF_Files/3701/alakhbar20190302.pdf'

In [20]:
response = requests.get(
    url='https://proxy.scrapeops.io/v1/',
    params={
        'api_key': '45869223-31aa-4433-9a78-40a03dcc05dd',
        'url': url,
    },
    )
response.status_code

200

In [16]:

with open('file.pdf', 'wb') as f:
    f.write(response.content)
    
pdf = PdfFileReader(open('file.pdf', 'rb'))
texts = {}

for page in range(1,pdf._get_num_pages()):
    with open('file.pdf', 'rb') as f:
        laparams = LAParams(char_margin=1.0, line_margin=0.5, word_margin=0.1)
        outfp = StringIO()
        outfp.seek(0)
        extract_text_to_fp(f, outfp, laparams=laparams, page_numbers={page})
        outfp.seek(0)
        text = outfp.read()
        reshaper = ArabicReshaper(configuration=configuration)
        corrected_text = get_display(text)
        reshaped_text = reshaper.reshape(corrected_text)
        #reshaped_text = dediac_ar(reshaped_text).strip()
        #texts.append(reshaped_text)
        # print('-'*50, page)
        #print(reshaped_text)
        page_number = ''.join(x for x in reshaped_text[:4] if x in string.digits)
        section = ''.join(x for x in reshaped_text[:50] if x not in string.digits).strip().replace('\n',' ').replace('ـ', '')
        section = [x for x in section.split() if len(x) > 2]
        if 'ﻟﺒﻨﺎﻥ' in section or 'ﺳﻴﺎﺳﺔ' in section or 'ﻣﺠﺘﻤﻊ' in section or 'ﺍﻗﺘﺼﺎﺩ' in section:
            print(page_number,'-' ,page)
            texts[page] = reshaped_text.strip().replace('\n',' ').replace('ـ', '').replace('�', '')
                # pdf_name = url[-12:-4] + '-page(' + page_number + '-' + str(int(page_number)-1)+').pdf'
                # output_pdf = PdfFileWriter()
                # output_pdf.addPage(pdf.getPage(page))
                # with open(pdf_name, 'wb') as f:
                #     output_pdf.write(f)
                
            
            
            
        #corr = Corrector().
        #corrected = corr.contextual_correct(reshaped_text)
        
        #texts[page_number] = reshaped_text.strip().replace('\n',' ').replace('ـ', '')

        # keywords = ['عتصم', 'حتج']
        # for keyword in keywords:
            
        #regex = re.compile('|'.join(keywords), re.IGNORECASE)
        #match = regex.search(reshaped_text)
        #if any(keyword in reshaped_text for keyword in keywords):
            #if keyword in keywords:
            #if regex.search(reshaped_text):
                #new_file_name = match.group() + '.pdf'
                # output_pdf = PdfFileWriter()
                # output_pdf.addPage(pdf.getPage(page))
                # with open('blabla.pdf', 'wb') as f:
                #     output_pdf.write(f)
                    #print(page_number)
            # else:
            #     print('not_found')
                # print(page_number)

#texts

PdfReadError: PDF starts with '{"sta', but '%PDF-' expected

In [54]:
texts[10]

'11  ﻣﺠﺘﻤﻊ  ﺍﻟﺨﻤﻴﺲ  26  ﻛﺎﻧﻮﻥ ﺍﻟﺜﺎﻧﻲ  2012  ﺍﻟﻌﺪﺩ  1619  ﻣﺘﻔﺮﻗﺎﺕ  ﻗﺘﻴﺎﻟﻦ ﻓﻲ ﺣﺎﺩﺙ ﺳﻴﺮ ﻋﻠﻰ ﺃﻭﺗﻮﺳﺘﺮﺍﺩ ﺍﻟﺒﺘﺮﻭﻥ  ﺗﻮﻓﻲ ﺃﻣﺲ ﺍﻣﻠﻮﺍﻃﻨﺎﻥ ﺟﻮﺭﺝ ﻃﺮﺑﻴﻪ ﻭﺯﻭﺟﺘﻪ ﻣﺎﺭﻱ ﺑﻄﺮﺱ ﺍﻟﺨﻮﺭﻱ  ﻃﺮﺑﻴﻪ )ﻣﻮﺍﻟﻴﺪ ﺩﻳﺮ ﺍﺃﻟﺤﻤﺮ( ﻓﻲ ﺣﺎﺩﺙ ﺳﻴﺮ ﺑﻌﺪ ﺍﺻﻄﺪﺍﻡ ﺳﻴﺎﺭﺗﻬﻤﺎ  ﻣﻦ ﻧﻮﻉ »ﺟﻴﺐ ﺷﻴﺮﻭﻛﻲ« ﺑﺸﺎﺣﻨﺔ »ﺑﻮﻛﻠﻨﻲ« ﺗﻌﻤﻞ ﻓﻲ ﻭﺭﺷﺔ ﺣﻔﺮﻳﺎﺕ  ﻋﻠﻰ ﺟﺎﻧﺐ ﺃﻭﺗﻮﺳﺘﺮﺍﺩ ﺍﻟﺒﺘﺮﻭﻥ. ﻳﺬﻛﺮ ﺃﻥ ﺟﻮﺭﺝ ﺗﻮﻓﻲ ﻋﻠﻰ ﺍﻟﻔﻮﺭ، ﺃﻣﺎ  ﻣﺎﺭﻱ ﻓﻜﺎﻧﺖ ﺇﺻﺎﺑﺘﻬﺎ ﺧﻄﺮﺓ ﻭﻋﻤﻠﺖ ﻋﻨﺎﺻﺮ ﺍﻟﺼﻠﻴﺐ ﺍﺃﻟﺤﻤﺮ ﻭﺍﻟﺪﻓﺎﻉ  ﺍﻣﻠﺪﻧﻲ ﻋﻠﻰ ﺇﺧﺮﺍﺟﻬﺎ ﻣﻦ ﺍﻟﺴﻴﺎﺭﺓ ﺑﺼﻌﻮﺑﺔ. ﻭﻗﺪ ﺃﻭﻗﻔﺖ ﺍﺃﻟﺠﻬﺰﺓ  ﺍﺃﻟﻤﻨﻴﺔ ﺳﺎﺋﻖ ﺍﻟﺸﺎﺣﻨﺔ ﺧﺎﻟﺪ ﺗﻮﻓﻴﻖ ﻋﻠﻢ ﺍﻟﺪﻳﻦ ﻣﻦ ﺍﻣﻠﻨﻴﺔ، ﻭﺑﻮﺷﺮﺕ  ﺍﻟﺘﺤﻘﻴﻘﺎﺕ ﻓﻲ ﺍﻟﺤﺎﺩﺙ.  ﺃﺻﺤﺎﺏ ﺍﺃﻟﻤﺮﺍﺽ ﺍﻣﻠﺴﺘﻌﺼﻴﺔ ﻳﻌﺘﺼﻤﻮﻥ ﺃﻣﺎﻡ »ﺍﺃﻟﻮﻧﺮﻭﺍ«   ﻣﺖ »ﺍﻟﺠﺒﻬﺔ ﺍﻟﺪﻳﻤﻘﺮﺍﻃﻴﺔ ﻟﺘﺤﺮﻳﺮ  ﻧﻆ ﻓﻠﺴﻄﻨﻲ« ﻭ»ﻟﺠﺎﻥ ﺣﻖ ﺍﻟﻌﻮﺩﺓ« ﻭ»ﻟﺠﻨﺔ  ﺍﻣﻠﺘﺎﺑﻌﺔ ﺃﻟﺼﺤﺎﺏ ﺍﺃﻟﻤﺮﺍﺽ ﺍﻣﻠﺴﺘﻌﺼﻴﺔ  ﻭﺍﻟﺪﺍﺋﻤﺔ«، ﺍﻋﺘﺼﺎﻣﺎ ﻭﻣﺆﺗﻤﺮﺍ ﺻﺤﺎﻓﻴﺎ  ﺃﻣﺎﻡ ﻋﻴﺎﺩﺓ »ﺍﺃﻟﻮﻧﺮﻭﺍ« ﻓﻲ ﻣﺨﻴﻢ ﺍﻟﺒﺪﺍﻭﻱ،  ﺣﻀﺮﻩ ﺃﺻﺤﺎﺏ ﺍﺃﻟﻤﺮﺍﺽ ﺍﻣﻠﺴﺘﻌﺼﻴﺔ  ﻓﻲ ﻣﺨﻴﻢ ﺍﻟﺒﺪﺍﻭﻱ، ﻭﺍﻣﻠﺘﻀﺮﺭﻭﻥ ﻣﻦ  ﻣﺸﺮﻭﻉ ﺗﺮﻣﻴﻢ ﺍﻟﺒﻴﻮﺕ ﻓﻴﻪ، ﻭﻣﻬﺞ ﺭﻭ  ﻟﻮ ﺍﻟﻔﺼﺎﺋﻞ ﻭﺍﻟﻠﺠﺎﻥ  ﺗﻞ ﺍﻟﺰﻋﺘﺮ، ﻭﻣﻤﺚ ﺍﻟﺸﻌﺒﻴﺔ ﻭﻣﺆﺳﺴﺎﺕ ﺃﻫﻠﻴﺔ. ﻭﺭﻓﻊ ﺍﻣﻠﻌﺘﺼﻤﻮﻥ ﺍﻟﺸﻌﺎﺭﺍﺕ ﺍﻟﺘﻲ ﺗﻄﺎﻟﺐ  ﺍﻟﻮﻛﺎﻟﺔ »ﺑﺰﻳﺎﺩﺓ ﻣﻮﺍﺯﻧﺘﻬﺎ ﺍﻟﺼﺤﻴﺔ ﻭﺭﻓﻊ ﺍﻟﺘﻐﻄ

In [38]:
pdf.getPage(1)

{'/ArtBox': [0, 0, 1644.09, 1247.24],
 '/BleedBox': [0, 0, 1644.09, 1247.24],
 '/Contents': {'/Filter': '/FlateDecode'},
 '/CropBox': [0, 0, 1644.09, 1247.24],
 '/MediaBox': [0, 0, 1644.09, 1247.24],
 '/Parent': {'/Type': '/Pages',
  '/Count': 1,
  '/Kids': [IndirectObject(3, 0, 140600867993728)]},
 '/Resources': {'/ColorSpace': {'/CS0': ['/ICCBased',
    IndirectObject(11, 0, 140600846506688)]},
  '/ExtGState': {'/GS0': {'/AIS': False,
    '/BM': '/Normal',
    '/CA': 1,
    '/OP': False,
    '/OPM': 1,
    '/SA': True,
    '/SMask': '/None',
    '/Type': '/ExtGState',
    '/ca': 1,
    '/op': False},
   '/GS1': {'/AIS': False,
    '/BM': '/Normal',
    '/CA': 1,
    '/OP': True,
    '/OPM': 0,
    '/SA': True,
    '/SMask': '/None',
    '/Type': '/ExtGState',
    '/ca': 1,
    '/op': True}},
  '/Font': {'/T1_0': {'/BaseFont': '/KXPBOY+Amine-Bold',
    '/Encoding': {'/BaseEncoding': '/WinAnsiEncoding',
     '/Differences': [1,
      '/uniFE8B',
      '/uniFEDE',
      '/uniFE97',
    

In [28]:
import unicodedata

In [78]:
word = 'أبجدهوزحطيكلمنسعفصقرشتثخذضظغ'

In [79]:
arabic_alpha = {}

In [30]:
def unicodes(string):
     return ' '.join('U+{:04X}'.format(ord(c)) for c in string)

In [35]:
nfd_example = unicodedata.normalize("NFD", word)
nfd_example = unicodes(nfd_example)

In [36]:
nfd_example.encode("WINDOWS-1252", errors='ignore')

b'U+FE8D U+FEDF U+FE92 U+FE98 U+FEAE U+FEED U+FEE5'

In [111]:
print(nfd_example.encode('WINDOWS-1252', 'namereplace'))

b'U+FE8D U+FEDF U+FE92 U+FE98 U+FEAE U+FEED U+FEE5'


In [113]:
'ﺑﺴﺮﻋﺔ'.encode('WINDOWS-1251',errors='namereplace')

b'\\N{ARABIC LETTER BEH INITIAL FORM}\\N{ARABIC LETTER SEEN MEDIAL FORM}\\N{ARABIC LETTER REH FINAL FORM}\\N{ARABIC LETTER AIN INITIAL FORM}\\N{ARABIC LETTER TEH MARBUTA FINAL FORM}'

In [86]:
d = d.replace("{",' ')
d

'ALEF BEH JEEM DAL HEH WAW ZAIN HAH TAH YEH KAF LAM MEEM NOON SEEN AIN FEH SAD QAF REH SHEEN TEH THEH KHAH THAL DAD ZAH GHAIN'

In [87]:
d = d.split()
d

['ALEF',
 'BEH',
 'JEEM',
 'DAL',
 'HEH',
 'WAW',
 'ZAIN',
 'HAH',
 'TAH',
 'YEH',
 'KAF',
 'LAM',
 'MEEM',
 'NOON',
 'SEEN',
 'AIN',
 'FEH',
 'SAD',
 'QAF',
 'REH',
 'SHEEN',
 'TEH',
 'THEH',
 'KHAH',
 'THAL',
 'DAD',
 'ZAH',
 'GHAIN']

In [88]:
word = [x for x in word]

In [89]:
for i, j in zip(word, d):
    arabic_alpha[j] = i

In [90]:
arabic_alpha

{'ALEF': 'أ',
 'BEH': 'ب',
 'JEEM': 'ج',
 'DAL': 'د',
 'HEH': 'ه',
 'WAW': 'و',
 'ZAIN': 'ز',
 'HAH': 'ح',
 'TAH': 'ط',
 'YEH': 'ي',
 'KAF': 'ك',
 'LAM': 'ل',
 'MEEM': 'م',
 'NOON': 'ن',
 'SEEN': 'س',
 'AIN': 'ع',
 'FEH': 'ف',
 'SAD': 'ص',
 'QAF': 'ق',
 'REH': 'ر',
 'SHEEN': 'ش',
 'TEH': 'ت',
 'THEH': 'ث',
 'KHAH': 'خ',
 'THAL': 'ذ',
 'DAD': 'ض',
 'ZAH': 'ظ',
 'GHAIN': 'غ'}

In [107]:
pdf_word = 'ﺍﻟﺒﺘﺮﻭﻥ'
d = pdf_word.encode('WINDOWS-1251',errors='namereplace')
word_list = str(d).split()
word_list = [x for x in word_list if x in arabic_alpha.keys()]
corrected_word = ''
for x in word_list:
    corrected_word = ''.join(corrected_word + arabic_alpha[x])
corrected_word

'ألبترون'

In [114]:
'محاولة' == 'محاولة'

True

In [101]:
corrected_word

['أ', 'ب', 'و', 'ل', 'ن', 'ر', 'ت']

In [103]:
word_list

['ALEF', 'LAM', 'BEH', 'TEH', 'REH', 'WAW', 'NOON']

In [16]:
keywords = ['عتصم', 'حتج', 'ﺃﺳﺒﻮﻉ']
#regex = re.compile('|'.join(keywords), re.IGNORECASE)
#print(texts)
pdf = PdfFileReader(open('file.pdf', 'rb'))
for page in range(0,pdf._get_num_pages()):
    for key, value in texts.items():
        #match = regex.search(value)
        #print(match)
        words = value.split()
        #print(words)
        for keyword in keywords:
            for word in words:
                if keyword in word:
                    print(key, word)
                    # output_pdf = PdfFileWriter()
                    # output_pdf.addPage(pdf.getPage(page))
                    # with open('single_odf.pdf', 'wb') as f:
                    #     output_pdf.write(f)

32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 ﺃﺳﺒﻮﻉ،
17 ﺃﺳﺒﻮﻉ
23 »ﺃﺳﺒﻮﻉ
32 ﺃﺳﺒﻮﻉ
11 ﺃﺳﺒﻮﻉ
15 ﺃﺳﺒﻮﻉ،
15 

In [13]:
corr = Corrector()
corr.spell_correct('ﺇﻟﻨﺸﺎء ',2)

'ﺇﻟﻨﺸﺎﺀ '

In [63]:
import codecs

In [67]:
pdf_string = "ﺍﻟﺴﻨﺔﺍﻟﺮﺍﺑﻌﺔﻋﺸﺮﺓ"
ex= "السنةالرابعةعشرة"
pdf_string_encoded = codecs.decode(codecs.encode(pdf_string))
ex_encoded = codecs.decode(codecs.encode(ex))
#pdf_string_decoded = pdf_string_encoded.decode("cp1251")

print(pdf_string_encoded)
print(ex_encoded)

if pdf_string_encoded == ex_encoded:
    print("Strings match!")
else:
    print("Strings don't match.")
# def clean_text(text):
#     text = text.strip()
#     text = text.replace('\n', '')
#     text = text.replace('\r', '')
#     text = text.replace('\t', '')
#     return text
# cleaned_text = clean_text(pdf_string)
# cleaned_text = pdf_string.encode('utf-8')
# expected_string_encoded = expected_string.encode('utf-8')

ﺍﻟﺴﻨﺔﺍﻟﺮﺍﺑﻌﺔﻋﺸﺮﺓ
السنةالرابعةعشرة
Strings don't match.


In [28]:
codecs. .getdecoder('utf-8')

<function encodings.utf_8.decode(input, errors='strict')>

In [23]:
encodings = ["utf-16-le", "utf-16-be", "utf-8", "ascii"]

for encoding in encodings:
    try:
        decoded_string = codecs.decode(pdf_string, encoding)
        print("The original encoding is likely:", encoding)
        print("Decoded text:", decoded_string)
        break
    except:
        print("Failed to decode using", encoding)

The original encoding is likely: utf-16-le
Decoded text: 뫯龻뫯ꢻ뫯趺믯꺺뫯醺믯钺믯뢺뫯鎺


In [89]:
from charset_normalizer import from_path

results = from_path('./file.pdf')

print(str(results.best()))

None


In [98]:
string1 = pdf_string.encode()
string2 = ex.encode()

if string1.decode('Windows-1252') == string2.decode('Windows-1252'):
    unified_string = string1.decode('Windows-1252').encode('utf-8')
    print("Strings are the same")
else:
    print("Strings are different")

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in position 2: character maps to <undefined>

In [59]:
'ﺧﺮﺍﻓﺔ' == 'خرافة'

False

In [35]:
keywords = ['ﺗﺮﻛ ', 'asdasd', 'ghghgh']

In [36]:
for x in texts['32'].split():
    for key in keywords:
        if key in x.strip():
            print(x)

In [26]:
keywords = ['عتصم', 'حتج', 'ﺗﻨﻤﻮﻱ']
regex = re.compile('|'.join(keywords), re.IGNORECASE)

for keyword in keywords:
    
    for key, value in texts.items():
        #match = regex.search(value)
        if keyword in value:
            output_pdf = PdfFileWriter()
            output_pdf.addPage(pdf.getPage(int(key)))
            with open('single_odf.pdf', 'wb') as f:
                output_pdf.write(f)
                print('yes')
        # else:
        #     print('Not_found')

yes


In [17]:
response = requests.get(url)

with open('file.pdf', 'wb') as f:
    f.write(response.content)

with open('file.pdf', 'rb') as f:
    laparams = LAParams(char_margin=1.0, line_margin=0.5, word_margin=0.1)
    outfp = StringIO()
    outfp.seek(0)
    for page in range(0, 10):
        outfp.seek(0)
        extract_text_to_fp(f, outfp, laparams=laparams, page_numbers={page})
        outfp.seek(0)
        text = outfp.read()
        reshaper = ArabicReshaper(configuration=configuration)
        corrected_text = get_display(text)
        reshaped_text = reshaper.reshape(corrected_text)
        # print('-'*50, page)
        #print(reshaped_text)
        
        if 'ﺃﻟﻐﺎﺛﺎ ﻛﺮﻳﺴﺘﻲ'in reshaped_text:
            pdf = PdfFileReader(f)
            output_pdf = PdfFileWriter()
            output_pdf.addPage(pdf.getPage(page))
            with open('single_page.pdf', 'wb') as f:
                output_pdf.write(f)
        else:
            print('not_found')

32 ﺻﻔﺤﺔ
1000 ﻟﻴﺮﺓ

ﺍﺍﻟﺜﻨﻴﻦ  2  ﻛﺎﻧﻮﻥ ﺍﺃﻟﻮﻝ  2019
ﺍﻟﻌﺪﺩ  3922  ﺍﻟﺴﻨﺔ  ﺍﻟﺮﺍﺑﻌﺔ ﻋﺸﺮﺓ
lundi  2  Décembre  2019  no 3922   14ème année

ﺭﺃﺱ ﺍﻟﻤﺎﻝ
ﺍﺣﺘﻴﺎﻁ »ﺍﻟﻤﺮﻛﺰﻱ« 
ﺑﺎﻟﺪﻭﺍﻟﺮ ﻳﺘﺎﻟﺸﻰ
● ﻣﺤﻤﺪ ﻭﻫﺒﺔ
ﻧﻤﻮﺫﺝ ﺍﻗﺘﺼﺎﺩﻱ
ﺹ ﻡ ﻡ ﻟﻘﺘﻞ ﺍﻟﺰﺭﺍﻋﺔ

● ﻋﺒﺪ ﺍﻟﺤﻠﻴﻢ ﻓﻀﻞ ﷲ
ﻋﻦ ﺧﺮﺍﻓﺔ ﺍﻟﻨﻤﻮﺫﺝ
ﻭﺣﻜﺎﻳﺔ ﺍﻧﻬﻴﺎﺭ ﻣﺆﺝ ﻝ

ﺃﺳﺒﻮﻉ ﺣﺎﺳﻢ ﺣﻜﻮﻣﻴﺎ  ]2[

ﺍﻟﻔﻮﺿﻰ 
ﺗﻀﺮﺏ ﺍﻟﻌﺮﺍﻕ

]14 ـ  15[

)
ﺃ

ﻑ
ﺏ

(

ﺗﺮﻛﻴﺎ ﻭﻟﻴﺒﻴﺎ ﻭﻏﺎﺯ 
ﺍﻟﻤﺘﻮﺱ ﻁ
»ﺍﺁﻟﻦ ﺑﺪﺃﺕ ﺍﻟﻠﻌﺒﺔ«!

18

ﻗﻀﻴﺔ

ﺍﻟﻴﻤﻦ

ﻣﻘﺎﺑﻠﺔ

 ﺳﻌﻮﺩﻱ  
ﺿﻐﻂ
ﺇﻟﻨﺸﺎﺀ ﻣﻨﻄﻘﺔ 
ﻋﺎﺯﻟﺔ
16

ﺇﻳﻠﻲ ﻣﺸﻨﺘﻒ
ﻧﺘﻤﻨﻰ ﺃﻥ ﻳﺒﻘﻰ 
ﺍﻟﺒﻠﺪ ﻟﺘﺒﻘﻰ ﺍﻟﻠﻌﺒﺔ

8


 
 

not_found
3

ﻟﺒﻨﺎﻥ

ﺍﺇﻟﺜﻨﻴﻦ  2  ﻛﺎﻧﻮﻥ ﺍﺃﻟﻮﻝ  2019  ﺍﻟﻌﺪﺩ  3922

ﻋﺖ 

ﺭ ﻭﺧﻂ

ﻗﺔ ﺑﺄﺳﻤﺎﺀ ﻫﺬﻩ ﺍﻟﺠﻤﻌﻴﺎﺕ 

ﻡ ﻭﻓﺮﺹ ﺍﻟﻌﻤﻞ ﻭﺍﻟﺪﻭﺭ ﺍﻟﻘﻴﺎﺩﻱ، ﻭﺭﻋﺎﻳﺔ 

ﻭﻣﺎ ﺍﻟﺬﻱ ﺗﻤﻠﻜﻪ ﺍﺁﻟﻦ، ﻭﺻﻮﺭﺓ ﻣﺒﺲ ﻃﺔ ﻋﻦ ﻧﻈﺎﻡ ﺣﻴﺎﺗﻬﺎ ﺍﻟﻌﺎﻣﺔ، 
ﻣﺜﻞ ﺃﻳﻦ ﺗﻌﻴﺶ ﻭﺃﻳﻦ ﻳﺘﻌﻠﻢ ﺍﺃﻟﺒﻨﺎﺀ ﻭﺃﻳﻦ ﻳﻘﻀﻮﻥ ﺍﺇﻟﺠﺎﺯﺍﺕ ﻭﻧﻮﻉ 
ﺍﻟﺴﻴﺎﺭﺍﺕ ﻭﺍﻟﺨﺪﻣﺎﺕ ﺍﻟﺘﻲ ﻳﺤﺼﻠﻮﻥ ﻋﻠﻴﻬﺎ ﻓﻲ ﺣﻴﺎﺗﻬﻢ ﺍﻟﻴﻮﻣﻴﺔ؟
، ﻣﻦ ﺃﻳﻦ ﻱ ﻧﻔﻖ ﺣﺰﺏ ﺍﻟﻜﺘﺎﺋﺐ ﻃﻮﺍﻝ ﺍﻟﻌﻘﻮﺩ ﺍﻣﻼﺿﻴﺔ، ﻭﻣﺎ ﻫﻲ 
ﻣﺜﺎﻝ
ﺍﻣﻠﻬﻦ ﺍﻟﺘﻲ ﻳﻘﻮﻡ ﺑﻬﺎ ﺁﻝ ﺍﻟﺠﻤﻴﻞ ﺑﻜﻞ ﻓﺮﻭﻋﻬﻢ؟ ﺃﻭ ﻟﻨﺴﺄﻝ ﺣﺰﺏ 
ﻓﻘﺮﺍﺀ ﺍﻣﻠﺴﻴﺤﻴﻨﻲ، ﺃﻱ ﺍﻟﻘﻮﺍﺕ ﺍﻟﻠﺒﻨﺎﻧﻴﺔ، ﻋﻦ ﻣﺼﺪﺭ ﻧﻔﻘﺎﺗﻪ ﺍﻟﺸﻬﺮﻳ

In [60]:

# pdf_response = requests.get(url)

# with open('file.pdf', 'wb') as f:
#     f.write(pdf_response.content)

# with open('file.pdf', 'rb') as f:
    # pdf = PDFQuery(f)
    # pdf.load()
    # blocks = pdf.extract([
    #     ('with_parent','LTPage'),
    #     ('block_txt','LTTextLineHorizontal')
    # ])
    # for block in blocks:
    #     print(block.text.strip())
    # pdf = PdfFileReader(f)

    # outfp = StringIO()
    # laparams = LAParams(char_margin=1.0, line_margin=0.5, word_margin=0.1)
    
    # for page in range(1,pdf.getNumPages()):
    #     outfp.seek(0)
    #     extract_text_to_fp(f, outfp, laparams=laparams, page_numbers={page})
    #     outfp.seek(0)
    #     text = outfp.read()
    #     reshaper = ArabicReshaper(configuration=configuration)
    #     #corrected_text = reshaper.reshape(text)
    #     corrected_text = get_display(text)
    #     print(corrected_text)
    #     if 'أسبوع' in corrected_text:
    #         print('TADAAAAAAAAAAAAAAA',page)
    #         break


3

لبنان

اإلثنين  2  كانون األول  2019  العدد  3922

عت 

ر وخط

قة بأسماء هذه الجمعيات 

م وفرص العمل والدور القيادي، ورعاية 

وما الذي تملكه اآلن، وصورة مبسّ طة عن نظام حياتها العامة، 
مثل أين تعيش وأين يتعلم األبناء وأين يقضون اإلجازات ونوع 
السيارات والخدمات التي يحصلون عليها في حياتهم اليومية؟
، من أين يُ نفق حزب الكتائب طوال العقود املاضية، وما هي 
مثال
املهن التي يقوم بها آل الجميل بكل فروعهم؟ أو لنسأل حزب 
فقراء املسيحيني، أي القوات اللبنانية، عن مصدر نفقاته الشهرية 
والسنوية في كل القطاعات، بما في ذلك كلفة حياة قائده 
وخدماته وحراسته. واألمر نفسه بالنسبة إلى التيار الوطني 
الحر وحركة أمل، والزعيم الوطني الكبير املفدّ ى وليد جنبالط، 
وحجم اإلنفاق القائم على الدولة لحراستهم وتأمني خدماتهم 
اللوجستية؟. وماذا عن بقية الزعامات، من بقايا اإلقطاع 
األسعدي في الجنوب، إلى مراعبة عكار، مرورًا بآل فرنجية 
ومعوض والخازن وفرعون والصحناوي، حتى آل الحريري في 
هذه األيام...؟
ماذا عن كل جمعيات دعم املرأة وتمكينها وحريتها ورفض 
تعنيفها، ومقاومة التمييز الجندري، وحق األمومة والجنسية، 
واملساواة

In [58]:
# text_ls = [] 
# for link in pdf_links:
#     # download the PDF
#     pdf_url = link['href']
    
#     pdf_content = pdf_response.content
#     # extract the text from the PDF
#     text = extract_text_from_pdf(pdf_content)
#     text_ls.append(text)
    
# print(soup)

In [59]:
# url = df.iloc[0]['pdf_link']
# pdf_response = requests.get(url)
# pdf_content = pdf_response.content
# text = extract_text_from_pdf(pdf_content)
# text